In [5]:
pip install langchain_community unstructured --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 58.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 28.1 MB/s eta 0:00:00


In [14]:
pip install langchain --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.2 MB/s eta 0:00:00


In [18]:
pip install langchain_google_vertexai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 896.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 9.5 MB/s eta 0:00:00


In [24]:
pip install faiss-cpu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 23.6 MB/s eta 0:00:00


In [42]:
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.vectorstores import FAISS
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_vertexai import VertexAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ParentDocumentRetriever
import os
from langchain.chains.question_answering import load_qa_chain

In [7]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "lumen-b-ctl-047-e2aeb24b0ea0.json"

In [70]:
loader = UnstructuredExcelLoader("Jamaica_Mobile_KPI_ALL_20240509.xlsx", mode="elements")
docs = loader.load()
docs[0]

Document(page_content='\n\n\nLIME_Jamaica\n\n-38964.79\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n0\n\n\n', metadata={'source': 'Jamaica_Mobile_KPI_ALL_20240509.xlsx', 'filename': 'Jamaica_Mobile_KPI_ALL_20240509.xlsx', 'last_modified': '2024-05-10T13:52:05', 'page_name': 'Jamaica', 'page_number': 1, 'text_as_html': '<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>LIME_Jamaica</td>\n      <td></td>\n      <td>-38964.79</td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td></

In [71]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [72]:
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@003")

In [75]:
content = "\n\n".join(str(page.page_content) for page in docs)
texts = parent_splitter.split_text(content)

In [76]:
vector_store = FAISS.from_texts(texts, embeddings)

In [77]:
store = InMemoryStore()

In [78]:
retriever = ParentDocumentRetriever(
        vectorstore=vector_store,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter
    )

In [107]:
prompt_template = """
     Welcome to the role of telecom provider chatbot!
      Your role is to analyze KPI metrics and provide accurate responses based on the provided data.
      You're empowered to perform mathematical operations as needed to derive answers, ensuring precision without altering any numbers from the original dataset.

      Your goal is to deliver exact answers that correspond to the metrics outlined, assisting users with their inquiries.
      It's crucial to refrain from inventing numerical data and rely solely on the information provided.
      Let's leverage the insights from the KPI metrics to effectively support users with their telecom-related queries.
      ** do not use any outside numbers that is not provided to you**

     Context:\n {context}?\n\n
     Question: \n{question}\n\n

     Answer:
    """

In [108]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question", "chat_history"])
# memory = ConversationBufferMemory(memory_key='chat_history', input_key='question')
model = VertexAI(temperature=0.3, max_output_tokens=2048)

In [110]:
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [125]:
question = "Prepaid LTE Data Users Active 60 Day numbers for whole date range"

In [126]:
docs = retriever.get_relevant_documents(question)
response = chain({"input_documents": docs, "question": question}, return_only_outputs=True)

In [127]:
print(response['output_text'])

 The total number of Prepaid LTE Data Users Active 60 Day across the entire date range is 1,241,389.
